# 1. Set Up

In [1]:
!spack find -p tfel

-- linux-ubuntu20.04-x86_64 / gcc@9.4.0 -------------------------
tfel@4.0.0  /mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx
tfel@4.0.0  /mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-jjcwdu6cbil5dzqzjhjekn3jdzo3e6gc
==> 2 installed packages


In [2]:
# %env LD_LIBRARY_PATH=/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib

In [3]:
!echo "$(spack find -p tfel | awk '/\/mofem_install\// {print $NF "/lib"}')"

/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib
/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-jjcwdu6cbil5dzqzjhjekn3jdzo3e6gc/lib


In [4]:
import math
import os
import re
import sys
import time
import json
from pathlib import Path
import subprocess
import zipfile
import pydantic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import gmsh

sys.path.append('/mofem_install/jupyter/thomas/lateral_capacity_pipeline/src')

import setup
import core
import custom_models as cm
import utils as ut
import plotting
import calculations as calc
import postprocess
    
import matplotlib
matplotlib.rc('figure', figsize=(6, 6))

# 2. Simulation Parameters

In [5]:
#in MPa
def initialize_parameters(soil_model: cm.PropertyTypeEnum = cm.PropertyTypeEnum.le,**kwargs) -> cm.AttrDict:
    params = cm.AttrDict()
    setattr(params, "soil_model", soil_model)
    for key,value in kwargs.items():
        setattr(params, key, value)
    params.tester = cm.TestAttr(
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=500, poisson_ratio=0.3),
            cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(youngs_modulus=500,poisson_ratio=0.3, HardeningSlope = 2.5, YieldStress = 0.25),
            cm.PropertyTypeEnum.vM_Implicit_mfront: cm.VonMisesProperties(youngs_modulus=500, poisson_ratio=0.3, HardeningSlope = 2.5, YieldStress = 0.25),
            cm.PropertyTypeEnum.DP_Implicit_mfront: cm.DruckerPragerProperties(youngs_modulus=500, poisson_ratio=0.3, phi=np.radians(10), c=0.125, v=np.radians(0)),
            cm.PropertyTypeEnum.DP_HYPER_Implicit_mfront: cm.DPHYPERProps(youngs_modulus=500, poisson_ratio=0.3, phi=np.radians(10), c=0.125, v=np.radians(0), proximity = 1e-2),
            cm.PropertyTypeEnum.MCC: cm.CamClayProperties(),
            }, 
        
    )
 
    def custom_generate_physical_groups_mfront(params) -> list[cm.PhysicalGroup]:
        physical_group_list = [
            cm.PhysicalGroup(
                meshnet_id=1,
                name="MFRONT_MAT_1",
                preferred_model=params.tester.preferred_model,
                props=params.tester.props,
                group_type=cm.PhysicalGroupType.MATERIAL,
                ),
            # cm.PhysicalGroup(
            #     meshnet_id=7,
            #     name="MFRONT_MAT_2",
            #     preferred_model=params.tester.preferred_model,
            #     props=params.tester.props,
            #     group_type=cm.PhysicalGroupType.MATERIAL,
            #     ),
        ]
        return physical_group_list
    
    def custom_generate_physical_groups_mofem(params) -> list[cm.PhysicalGroup]:
        physical_group_list = []
        return physical_group_list
    
    if getattr(params, "custom_mesh_filepath", False):
        try:
            if params.use_mfront:
                params.custom_generate_physical_groups = custom_generate_physical_groups_mfront
            else:
                params.custom_generate_physical_groups = custom_generate_physical_groups_mofem
        except NameError as e:
            raise NameError("Must provided the custom callback 'custom_generate_physical_groups' to inject attributes to the main commmand.")
    
    
    params.dim = 3

    
    
    params.wk_dir = Path(f"/mofem_install/jupyter/thomas/lateral_capacity_pipeline")
    params.read_med_exe = "/mofem_install/jupyter/thomas/um_view/bin/read_med"
    params.h5m_to_vtk_converter = "/mofem_install/jupyter/thomas/um_view/bin/convert.py"
    params.partition_exe = "/mofem_install/jupyter/thomas/um_view/bin/mofem_part"
    if params.use_mfront:
        params.exe = f"/mofem_install/jupyter/thomas/um_view/tutorials/adv-1/contact_3d"
    else:
        params.exe = f"/mofem_install/jupyter/thomas/um_view_release/adolc_plasticity/adolc_plasticity_3d"
    params.paraview_path = "/mofem_install/jupyter/thomas/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython"
    params.ffmpeg_path = '/mofem_install/jupyter/thomas/ffmpeg-7.0.2-amd64-static/ffmpeg'
    params.preset_dir = '/mofem_install/jupyter/thomas/lateral_capacity_pipeline/src/presets'
    params.options_file = "/mofem_install/jupyter/thomas/lateral_capacity_pipeline/param_file.petsc"
    
    return params

In [6]:
params = initialize_parameters(
    nproc = 32, # number of processors/cores used
    order = 1, #order of approximation functions
    case_name = "test_3D_uniaxial",
    base = "hex",
    custom_mesh_filepath="/mofem_install/jupyter/thomas/lateral_capacity_pipeline/user_data/mesh/test/cube_Hm_adolc_uniaxial_NA.cub", 
    # soil_model=cm.PropertyTypeEnum.vM_Default_mfront,
    # soil_model=cm.PropertyTypeEnum.vM_Implicit_mfront,
    # soil_model=cm.PropertyTypeEnum.DP_Implicit_mfront,
    # soil_model=cm.PropertyTypeEnum.DP_HYPER_Implicit_mfront,
    # soil_model=cm.PropertyTypeEnum.vM_adolc,
    soil_model=cm.PropertyTypeEnum.Hm_adolc,
    use_mfront = False,
    final_time = 1, # [s]
    time_step = 1 / 40, # [s]
    
    options_file = "/mofem_install/jupyter/thomas/lateral_capacity_pipeline/param_file.petsc",
    
    save_gauss = 0,
    convert_gauss = 0,
    
    FEA_completed=False,
    days_since_epoch=160,
    sim_otd=19,
    
    # the prescribed force/dosplacement is always applied to the max_z surface
    # params.prescribed_force = cm.ForceBoundaryCondition(f_x=0,f_y=0,f_z=60),
    # prescribed_disp = cm.EdgeBoundaryCondition(disp_uz=1),
    prescribed_BC_name = "FIX_Z_0",
    
    time_history = cm.TimeHistory(history = {
        0: 0, 
        1/4: 0.001, 
        2/4: 0, 
        3/4: -0.001,
        1: 0.001,
        }
    ),
    # force_time_history = cm.TimeHistory(history = {
    #     0: 0, 
    #     1/4: 5, 
    #     2/4: 5, 
    #     1: 10,
    #     }
    # ),
    # displacement_time_history = cm.TimeHistory(history = {
    #     0: 0, 
    #     1/40: 4.83316293E-06, 
    #     2/40: 4.83316293E-06, 
    #     1: 0.02,
    #     }
    # ),
    # secondary_force_time_history = cm.TimeHistory(history = {
    #     0: 0, 
    #     1/40: 0.156, 
    #     1: 0.156,
    #     }
    # ),
    mesh_size = int(1),
    
    points_of_interest = [
        cm.Point(x=0.5,y=0.5,z=0.5),
        cm.Point(x=-0.5,y=-0.5,z=-0.5),
        cm.Point(x=-0.5,y=-0.5,z=0.5),
    ],
    line_of_interest = cm.Line(pt1=cm.Point(x=0.5,y=0.5,z=0.5), pt2=cm.Point(x=0.5,y=0.5,z=-0.5),),
) # 2. Simulation Parameters
params = setup.setup(params) # 3. Log paths and meta
if params.FEA_completed:
    # core.generate_mesh(params) # 4. Generate the mesh
    # core.mofem_compute(params) # 5. Running the analysis and export to .vtk file format
    # core.export_to_vtk(params)
    # postprocess.zip_vtks(params)
    # postprocess.extract_log(params)
    # postprocess.extract(params)
    postprocess.calculate(params)
    postprocess.plot(params)
    pass
else:
    core.generate_mesh(params) # 4. Generate the mesh
    core.mofem_compute(params) # 5. Running the analysis and export to .vtk file format
    core.export_to_vtk(params)
    postprocess.zip_vtks(params)
    postprocess.extract_log(params)
    postprocess.extract(params)
    postprocess.calculate(params)
    postprocess.plot(params)
    

╭───────────────────────────────╮
│  Simulation #27 for the day.  │
╰───────────────────────────────╯
╭─────────────────────────────────────╮
│  PARTITIONING MESH with mofem_part  │
╰─────────────────────────────────────╯
[0] <inform> MoFEM version 0.14.0 (MOAB 5.5.0 Petsc Release Version 3.16.6, Mar 30, 2022 )
[0] <inform> git commit id 693a01d8ad1e79f10b1dc4f1dd785f311161b1d1
[0] <inform> Local time: 2025-1-24 17:20:12
[0] <inform> UTC time: 2025-1-24 17:20:12
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316764 type BLOCKSET UNKNOWNNAME msId 1 name ADOLCMAT_1 block header:  blockCol = 0 blockMat = 0 blockDimension = 3
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316765 type BLOCKSET UNKNOWNNAME msId 2 name FIX_Z_0 block header:  blockCol = 0 blockMat = 0 blockDimension = 2
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316766 type BLOCKSET UNKNOWNNAME msId 3 name FIX_Z_1 block header:  blockCol = 0 blockMat = 0 blockD

<Figure size 600x600 with 0 Axes>